Ejecutar con GPU (10 minutos)

Instalando librerías

In [17]:
!pip install yfinance scikit-learn torch tqdm
!pip install onnxscript torch
!pip install tensorflow tf2onnx onnx onnxruntime
!pip install skl2onnx


Descargar la data en chunks de EUrUSD para 1hora de más de 10 k datos.

In [18]:
import pandas as pd
import yfinance as yf
import time
from datetime import datetime, timedelta
import os
from google.colab import drive

def download_chunk(symbol, interval, chunk_start, chunk_end, retries=3):
    """Descargar un chunk de datos con reintentos"""
    for attempt in range(retries):
        try:
            print(f"  📥 Intento {attempt + 1}/{retries}: {chunk_start.date()} → {chunk_end.date()}", end=" ")

            ticker = yf.Ticker("EURUSD=X")
            df = ticker.history(period="max", interval="1d")

            if df is None or len(df) == 0:
                print("❌ Sin datos")
                if attempt < retries - 1:
                    time.sleep(2)
                    continue
                return None

            # Limpiar datos
            df = df.reset_index()
            df.columns = [str(col).lower() for col in df.columns]

            if 'date' in df.columns:
                df.rename(columns={'date': 'time'}, inplace=True)
            elif 'datetime' in df.columns:
                df.rename(columns={'datetime': 'time'}, inplace=True)

            print(f"✅ {len(df)} velas")
            return df

        except Exception as e:
            print(f"❌ Error: {str(e)[:50]}")
            if attempt < retries - 1:
                time.sleep(3)
            else:
                return None

def download_historical_data(symbol, interval, start_date, end_date=None, chunk_days=90):
    """Descargar datos históricos en chunks"""
    if end_date is None:
        end_date = datetime.now()

    total_days = (end_date - start_date).days
    print(f"\n📊 Período: {start_date.date()} a {end_date.date()} ({total_days} días)")

    # Crear chunks
    chunks = []
    current = start_date
    while current < end_date:
        chunk_end = min(current + timedelta(days=chunk_days), end_date)
        chunks.append((current, chunk_end))
        current = chunk_end + timedelta(days=1)

    print(f"📦 Dividido en {len(chunks)} chunks de ~{chunk_days} días\n")

    all_data = []
    successful = 0

    for i, (chunk_start, chunk_end) in enumerate(chunks):
        print(f"Chunk {i+1}/{len(chunks)}:")

        chunk_data = download_chunk(symbol, interval, chunk_start, chunk_end)

        if chunk_data is not None and len(chunk_data) > 0:
            all_data.append(chunk_data)
            successful += 1
            print(f"  ✔ Progreso: {successful}/{len(chunks)} exitosos | Total velas: {sum(len(d) for d in all_data)}\n")
        else:
            print(f"  ✗ Chunk falló\n")

        time.sleep(1)

    # Combinar datos
    if all_data:
        print("🔄 Combinando datos...")
        combined = pd.concat(all_data, ignore_index=True)

        if 'time' in combined.columns:
            combined.sort_values('time', inplace=True)
            combined.drop_duplicates('time', inplace=True)
            combined.reset_index(drop=True, inplace=True)

            print(f"✅ Total: {len(combined)} velas")
            print(f"📅 Desde: {combined['time'].min()}")
            print(f"📅 Hasta: {combined['time'].max()}")

            return combined
        else:
            print("❌ Error: No se encontró columna 'time'")
            return None
    else:
        print("❌ No se descargó ningún dato")
        return None

def main():
    print("=" * 60)
    print("         DESCARGA INCREMENTAL DE DATOS FOREX - EURUSD")
    print("=" * 60)

    # Montar Drive
    try:
        drive.mount('/content/drive')
        print("✅ Drive montado\n")
    except:
        print("ℹ️ Drive ya estaba montado\n")

    # Configuración
    symbol = "EURUSD=X"
    interval = "1d"
    output_file = "/content/drive/MyDrive/EURUSD_1d_data.csv"

    print(f"⚙️ Configuración:")
    print(f"   Símbolo: {symbol}")
    print(f"   Intervalo: {interval}")
    print(f"   Archivo: {output_file}")

    # Verificar si el archivo existe
    if os.path.exists(output_file):
        print(f"\n📂 Archivo existente encontrado")
        print(f"📖 Leyendo datos existentes...")

        try:
            existing_df = pd.read_csv(output_file)
            existing_df['time'] = pd.to_datetime(existing_df['time'], utc=True)

            print(f"✅ {len(existing_df)} velas existentes")
            print(f"📅 Desde: {existing_df['time'].min()}")
            print(f"📅 Hasta: {existing_df['time'].max()}")

            # Calcular fecha de inicio para descarga incremental
            ultima_fecha = existing_df['time'].max()
            # Hacer fecha_actual timezone-aware
            fecha_actual = pd.Timestamp.now(tz='UTC')
            horas_diferencia = (fecha_actual - ultima_fecha).total_seconds() / 3600

            print(f"\n⏰ Diferencia con fecha actual: {horas_diferencia:.1f} horas")

            if horas_diferencia < 2:
                print(f"\n✅ Datos ya están actualizados")
                print(f"ℹ️ No se requiere descarga adicional\n")
                return

            # Descargar solo datos faltantes
            start_date = ultima_fecha + timedelta(hours=1)
            end_date = fecha_actual

            print(f"\n🔄 Descargando datos faltantes...")
            print(f"📅 Desde: {start_date}")
            print(f"📅 Hasta: {end_date}")

        except Exception as e:
            print(f"⚠️ Error leyendo archivo existente: {e}")
            print(f"🔄 Se procederá con descarga completa\n")
            existing_df = None
            days_back = 365*20
            start_date = datetime.now() - timedelta(days=days_back)
            end_date = datetime.now()
    else:
        print(f"\n📁 Archivo no existe - Descarga inicial completa")
        existing_df = None
        days_back = 365*20
        start_date = datetime.now() - timedelta(days=days_back)
        end_date = datetime.now()
        print(f"   Días atrás: {days_back}")

    # Test de conexión
    print(f"\n🔍 Probando conexión con Yahoo Finance...")
    try:
        test = yf.Ticker("EURUSD=X")
        info = test.info
        print(f"✅ Conexión OK - {info.get('longName', symbol)}")
    except Exception as e:
        print(f"⚠️ Advertencia: {str(e)[:100]}")

    # Descargar
    print("\n" + "="*60)
    print("INICIANDO DESCARGA...")
    print("="*60)

    new_data = download_historical_data(symbol, interval, start_date, end_date, chunk_days=90)

    if new_data is not None and len(new_data) > 0:
        # Combinar con datos existentes si los hay
        if existing_df is not None:
            print(f"\n🔗 Combinando con datos existentes...")
            combined_df = pd.concat([existing_df, new_data], ignore_index=True)
            combined_df.sort_values('time', inplace=True)
            combined_df.drop_duplicates('time', keep='last', inplace=True)
            combined_df.reset_index(drop=True, inplace=True)

            velas_nuevas = len(combined_df) - len(existing_df)
            print(f"✅ Datos combinados")
            print(f"   Total anterior: {len(existing_df):,}")
            print(f"   Velas nuevas: {velas_nuevas:,}")
            print(f"   Total actual: {len(combined_df):,}")

            final_df = combined_df
        else:
            final_df = new_data

        # Mostrar preview
        print("\n📋 Preview de los datos finales:")
        print(final_df.tail(10))
        print("\n📊 Estadísticas:")
        print(final_df.describe())

        # Guardar
        print(f"\n💾 Guardando en {output_file}...")
        try:
            final_df.to_csv(output_file, index=False)
            size_kb = os.path.getsize(output_file) / 1024
            print(f"\n{'='*60}")
            print(f"✅✅✅ ¡DESCARGA COMPLETADA EXITOSAMENTE! ✅✅✅")
            print(f"{'='*60}")
            print(f"📁 Archivo: {output_file}")
            print(f"📏 Tamaño: {size_kb:.2f} KB")
            print(f"📊 Total velas: {len(final_df):,}")
            print(f"📅 Desde: {final_df['time'].min()}")
            print(f"📅 Hasta: {final_df['time'].max()}")
            if existing_df is not None:
                print(f"🆕 Velas agregadas: {velas_nuevas:,}")
            print(f"{'='*60}\n")
        except Exception as e:
            print(f"\n❌ Error guardando: {e}")
    else:
        print("\n❌❌❌ DESCARGA FALLIDA ❌❌❌\n")

if __name__ == "__main__":
    main()

         DESCARGA INCREMENTAL DE DATOS FOREX - EURUSD
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive montado

⚙️ Configuración:
   Símbolo: EURUSD=X
   Intervalo: 1d
   Archivo: /content/drive/MyDrive/EURUSD_1d_data.csv

📂 Archivo existente encontrado
📖 Leyendo datos existentes...
✅ 5721 velas existentes
📅 Desde: 2003-12-01 00:00:00+00:00
📅 Hasta: 2025-12-17 00:00:00+00:00

⏰ Diferencia con fecha actual: 2.7 horas

🔄 Descargando datos faltantes...
📅 Desde: 2025-12-17 01:00:00+00:00
📅 Hasta: 2025-12-17 02:40:34.992548+00:00

🔍 Probando conexión con Yahoo Finance...
✅ Conexión OK - EUR/USD

INICIANDO DESCARGA...

📊 Período: 2025-12-17 a 2025-12-17 (0 días)
📦 Dividido en 1 chunks de ~90 días

Chunk 1/1:
  📥 Intento 1/3: 2025-12-17 → 2025-12-17 ✅ 5721 velas
  ✔ Progreso: 1/1 exitosos | Total velas: 5721

🔄 Combinando datos...
✅ Total: 5721 velas
📅 Desde: 2003-12-01 00:00:00+00:00
📅 Hasta: 2025-12-17 0

Actualización de la data

In [19]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import os
import time

def check_and_update_data():
    """Verificar y actualizar archivo CSV con datos más recientes"""

    print("=" * 60)
    print("      VERIFICACIÓN Y ACTUALIZACIÓN DE DATOS FOREX")
    print("=" * 60)

    # Configuración
    symbol = "EURUSD=X"
    interval = "1d"
    file_path = "/content/drive/MyDrive/EURUSD_1d_data.csv"
    min_required_rows = 5000

    # 1. Verificar si el archivo existe
    if not os.path.exists(file_path):
        print(f"\n❌ Error: No se encontró el archivo")
        print(f"📁 Ruta: {file_path}")
        print(f"\n💡 Ejecuta primero la celda de descarga inicial\n")
        return

    print(f"\n✅ Archivo encontrado")

    # 2. Cargar datos existentes
    print(f"📖 Leyendo datos existentes...")
    try:
        df = pd.read_csv(file_path)
        df['time'] = pd.to_datetime(df['time'], utc=True)
        print(f"✅ {len(df)} velas cargadas")
    except Exception as e:
        print(f"❌ Error leyendo archivo: {e}")
        return

    # 3. Verificar cantidad mínima de datos
    print(f"\n📊 Verificando cantidad de datos...")
    print(f"   Mínimo requerido: {min_required_rows:,}")
    print(f"   Actual: {len(df):,}")

    if len(df) < min_required_rows:
        print(f"❌ Datos insuficientes ({len(df):,} < {min_required_rows:,})")
        print(f"💡 Ejecuta la descarga inicial completa\n")
        return
    else:
        print(f"✅ Cantidad de datos suficiente")

    # 4. Verificar última fecha
    ultima_fecha = df['time'].max()
    fecha_actual = pd.Timestamp.now(tz='UTC')
    horas_diferencia = (fecha_actual - ultima_fecha).total_seconds() / 3600

    print(f"\n🕐 Verificando actualidad de los datos...")
    print(f"   Última vela: {ultima_fecha}")
    print(f"   Fecha actual: {fecha_actual}")
    print(f"   Diferencia: {horas_diferencia:.1f} horas")

    # 5. Decidir si actualizar
    if horas_diferencia < 1:
        print(f"\n✅ Datos actualizados (menos de 1 horas de diferencia)")
        print(f"ℹ️ No se requiere actualización\n")
        return

    print(f"\n⚠️ Datos desactualizados - Descargando datos faltantes...")

    # 6. Descargar datos nuevos
    start_date = ultima_fecha + timedelta(hours=1)
    end_date = fecha_actual

    print(f"\n📥 Descargando desde {start_date} hasta {end_date}...")

    try:
        ticker = yf.Ticker("EURUSD=X")
        df = ticker.history(period="max", interval="1d")

        if new_data is None or len(new_data) == 0:
            print(f"ℹ️ No hay datos nuevos disponibles")
            print(f"✅ El archivo ya está lo más actualizado posible\n")
            return

        # Limpiar datos nuevos
        new_data = new_data.reset_index()
        new_data.columns = [str(col).lower() for col in new_data.columns]

        if 'date' in new_data.columns:
            new_data.rename(columns={'date': 'time'}, inplace=True)
        elif 'datetime' in new_data.columns:
            new_data.rename(columns={'datetime': 'time'}, inplace=True)

        print(f"✅ {len(new_data)} nuevas velas descargadas")

        # 7. Combinar datos
        print(f"\n🔄 Combinando datos...")
        combined = pd.concat([df, new_data], ignore_index=True)
        combined.sort_values('time', inplace=True)
        combined.drop_duplicates('time', keep='last', inplace=True)
        combined.reset_index(drop=True, inplace=True)

        velas_agregadas = len(combined) - len(df)
        print(f"✅ Datos combinados")
        print(f"   Total velas: {len(combined):,}")
        print(f"   Nuevas velas agregadas: {velas_agregadas}")

        # 8. Guardar archivo actualizado
        print(f"\n💾 Guardando archivo actualizado...")
        combined.to_csv(file_path, index=False)
        size_kb = os.path.getsize(file_path) / 1024

        print(f"\n{'='*60}")
        print(f"✅✅✅ ¡ACTUALIZACIÓN COMPLETADA! ✅✅✅")
        print(f"{'='*60}")
        print(f"📁 Archivo: {file_path}")
        print(f"📏 Tamaño: {size_kb:.2f} KB")
        print(f"📊 Total velas: {len(combined):,}")
        print(f"🆕 Velas agregadas: {velas_agregadas}")
        print(f"📅 Desde: {combined['time'].min()}")
        print(f"📅 Hasta: {combined['time'].max()}")
        print(f"{'='*60}\n")

        # 9. Mostrar preview de datos nuevos
        if velas_agregadas > 0:
            print("📋 Preview de las últimas 5 velas:")
            print(combined.tail(5))
            print()

    except Exception as e:
        print(f"\n❌ Error durante la actualización: {e}\n")
        return

# Ejecutar
if __name__ == "__main__":
    check_and_update_data()

      VERIFICACIÓN Y ACTUALIZACIÓN DE DATOS FOREX

✅ Archivo encontrado
📖 Leyendo datos existentes...
✅ 5721 velas cargadas

📊 Verificando cantidad de datos...
   Mínimo requerido: 5,000
   Actual: 5,721
✅ Cantidad de datos suficiente

🕐 Verificando actualidad de los datos...
   Última vela: 2025-12-17 00:00:00+00:00
   Fecha actual: 2025-12-17 02:40:36.964923+00:00
   Diferencia: 2.7 horas

⚠️ Datos desactualizados - Descargando datos faltantes...

📥 Descargando desde 2025-12-17 01:00:00+00:00 hasta 2025-12-17 02:40:36.964923+00:00...

❌ Error durante la actualización: cannot access local variable 'new_data' where it is not associated with a value



Ahora el modelo en ONNX

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os
import time
from tqdm.auto import tqdm
import json
import joblib
#######################################################

start_total = time.time()


# ============================================================
#     LSTM MEJORADO CON EARLY STOPPING Y MÁS CAPAS
# ============================================================

class ForexDataset(Dataset):
    """Dataset personalizado para datos de Forex"""
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class ImprovedLSTMModel(nn.Module):
    """Modelo LSTM mejorado con más capas y arquitectura optimizada"""
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.3):
        super(ImprovedLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        x = self.fc1(last_output)
        x = self.relu(x)
        x = self.dropout(x)
        prediction = self.fc2(x)
        return prediction

class EarlyStopping:
    """Early stopping para prevenir overfitting"""
    def __init__(self, patience=10, min_delta=0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_epoch = 0

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"   ⚠️  EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f"\n   🛑 Early stopping activado en epoch {epoch}")
                    print(f"   📊 Mejor loss: {self.best_loss:.6f} en epoch {self.best_epoch}")
        else:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0

def prepare_data(df, sequence_length=60, features=['open', 'high', 'low', 'close']):
    """Preparar datos con MinMaxScaler"""
    print(f"\n📊 Preparando datos...")
    print(f"   Features: {features}")
    print(f"   Longitud de secuencia: {sequence_length}")

    data = df[features].values

    print(f"   ⏳ Normalizando datos con MinMaxScaler...")
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)
    print(f"   ✅ Normalización completada")
    print(f"   📐 Rango: [{data_scaled.min():.4f}, {data_scaled.max():.4f}]")

    print(f"   ⏳ Creando secuencias temporales...")
    X, y = [], []
    for i in tqdm(range(sequence_length, len(data_scaled)), desc="   Secuencias"):
        X.append(data_scaled[i-sequence_length:i])
        y.append(data_scaled[i, -1])

    X, y = np.array(X), np.array(y)

    print(f"\n✅ Datos preparados:")
    print(f"   Shape X: {X.shape}")
    print(f"   Shape y: {y.shape}")

    return X, y, scaler

def calculate_metrics(y_true, y_pred):
    """Calcular métricas de evaluación"""
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2,
        'MAPE': mape
    }

def train_model(model, train_loader, val_loader, epochs=100, learning_rate=0.001,
                device='cpu', early_stopping_patience=15):
    """Entrenar modelo con early stopping y métricas"""
    print(f"\n🏋️ Entrenando modelo...")
    print(f"   Epochs: {epochs}")
    print(f"   Learning rate: {learning_rate}")
    print(f"   Device: {device}")
    print(f"   Early stopping patience: {early_stopping_patience}")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    early_stopping = EarlyStopping(patience=early_stopping_patience, verbose=True)

    train_losses = []
    val_losses = []
    learning_rates = []
    best_model_state = None
    best_val_loss = float('inf')

    model.to(device)

    epoch_bar = tqdm(range(epochs), desc="📄 Epochs", position=0)

    for epoch in epoch_bar:
        start_time = time.time()
        current_lr = optimizer.param_groups[0]['lr']
        learning_rates.append(current_lr)

        # Training
        model.train()
        train_loss = 0
        train_bar = tqdm(train_loader, desc=f"  📚 Train {epoch+1}/{epochs}", leave=False, position=1)

        for X_batch, y_batch in train_bar:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions.squeeze(), y_batch)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item()
            train_bar.set_postfix({'loss': f'{loss.item():.6f}'})

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0
        val_bar = tqdm(val_loader, desc=f"  ✓ Val {epoch+1}/{epochs}", leave=False, position=1)

        with torch.no_grad():
            for X_batch, y_batch in val_bar:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                predictions = model(X_batch)
                loss = criterion(predictions.squeeze(), y_batch)
                val_loss += loss.item()
                val_bar.set_postfix({'loss': f'{loss.item():.6f}'})

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()

        scheduler.step(val_loss)

        epoch_time = time.time() - start_time

        epoch_bar.set_postfix({
            'train': f'{train_loss:.6f}',
            'val': f'{val_loss:.6f}',
            'lr': f'{current_lr:.6f}',
            'time': f'{epoch_time:.1f}s'
        })

        if (epoch + 1) % 10 == 0:
            print(f"\n   📊 Epoch [{epoch+1}/{epochs}]")
            print(f"      Train Loss: {train_loss:.6f}")
            print(f"      Val Loss: {val_loss:.6f}")
            print(f"      Learning Rate: {current_lr:.6f}")
            print(f"      Time: {epoch_time:.1f}s")

        early_stopping(val_loss, epoch + 1)
        if early_stopping.early_stop:
            print(f"\n   🏁 Entrenamiento detenido en epoch {epoch + 1}/{epochs}")
            break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\n   ✅ Cargado mejor modelo (val_loss: {best_val_loss:.6f})")

    print(f"\n✅ Entrenamiento completado")

    return train_losses, val_losses, learning_rates

def evaluate_model(model, test_loader, scaler, device='cpu'):
    """Evaluar modelo y calcular métricas"""
    print(f"\n🎯 Evaluando modelo...")

    model.eval()
    predictions = []
    actuals = []

    test_bar = tqdm(test_loader, desc="🧪 Evaluando")
    with torch.no_grad():
        for X_batch, y_batch in test_bar:
            X_batch = X_batch.to(device)
            pred = model(X_batch)
            predictions.extend(pred.cpu().numpy())
            actuals.extend(y_batch.numpy())

    predictions = np.array(predictions).reshape(-1, 1)
    actuals = np.array(actuals).reshape(-1, 1)

    n_features = scaler.n_features_in_
    pred_dummy = np.zeros((len(predictions), n_features))
    actual_dummy = np.zeros((len(actuals), n_features))

    pred_dummy[:, -1] = predictions.flatten()
    actual_dummy[:, -1] = actuals.flatten()

    pred_denorm = scaler.inverse_transform(pred_dummy)[:, -1]
    actual_denorm = scaler.inverse_transform(actual_dummy)[:, -1]

    metrics = calculate_metrics(actual_denorm, pred_denorm)

    print(f"\n📊 Métricas de evaluación:")
    print(f"   MAE:  {metrics['MAE']:.6f}")
    print(f"   MSE:  {metrics['MSE']:.6f}")
    print(f"   RMSE: {metrics['RMSE']:.6f}")
    print(f"   R²:   {metrics['R2']:.6f}")
    print(f"   MAPE: {metrics['MAPE']:.2f}%")

    return predictions, actuals, metrics, pred_denorm, actual_denorm

def plot_comprehensive_results(train_losses, val_losses, learning_rates,
                               predictions, actuals, pred_denorm, actual_denorm):
    """Crear visualizaciones completas de resultados con suavizado"""

    def smooth(data, weight=0.85):
        smoothed = []
        last = data[0]
        for point in data:
            smoothed_val = last * weight + (1 - weight) * point
            smoothed.append(smoothed_val)
            last = smoothed_val
        return smoothed

    fig = plt.figure(figsize=(20, 14))

    # 1. Training History
    ax1 = plt.subplot(3, 3, 1)
    ax1.plot(train_losses, label='Train Loss (raw)', linewidth=1, alpha=0.3)
    ax1.plot(smooth(train_losses), label='Train Loss (smooth)', linewidth=2)
    ax1.plot(val_losses, label='Val Loss (raw)', linewidth=1, alpha=0.3)
    ax1.plot(smooth(val_losses), label='Val Loss (smooth)', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=11)
    ax1.set_ylabel('Loss', fontsize=11)
    ax1.set_title('Training History (Smoothed)', fontsize=12, fontweight='bold')
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)

    # 2. Training History (Log Scale)
    ax2 = plt.subplot(3, 3, 2)
    ax2.plot(smooth(train_losses), label='Train Loss', linewidth=2)
    ax2.plot(smooth(val_losses), label='Val Loss', linewidth=2)
    ax2.set_xlabel('Epoch', fontsize=11)
    ax2.set_ylabel('Loss (log)', fontsize=11)
    ax2.set_title('Training History (Log Scale)', fontsize=12, fontweight='bold')
    ax2.set_yscale('log')
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)

    # 3. Learning Rate Schedule
    ax3 = plt.subplot(3, 3, 3)
    ax3.plot(learning_rates, linewidth=2.5, color='#FF6B6B')
    ax3.set_xlabel('Epoch', fontsize=11)
    ax3.set_ylabel('Learning Rate', fontsize=11)
    ax3.set_title('Learning Rate Schedule', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_yscale('log')

    # 4. Predictions vs Actuals (Normalized)
    ax4 = plt.subplot(3, 3, 4)
    sample_size = min(500, len(predictions))
    ax4.plot(actuals[:sample_size], label='Actual', linewidth=2, alpha=0.8, color='#4ECDC4')
    ax4.plot(predictions[:sample_size], label='Predicted', linewidth=2, alpha=0.8, color='#FF6B6B')
    ax4.set_xlabel('Sample', fontsize=11)
    ax4.set_ylabel('Price (normalized)', fontsize=11)
    ax4.set_title(f'Predictions vs Actuals (First {sample_size} samples)', fontsize=12, fontweight='bold')
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3)
    ax4.fill_between(range(sample_size), actuals[:sample_size].flatten(),
                      predictions[:sample_size].flatten(), alpha=0.2)

    # 5. Predictions vs Actuals (Denormalized)
    ax5 = plt.subplot(3, 3, 5)
    ax5.plot(actual_denorm[:sample_size], label='Actual', linewidth=2, alpha=0.8, color='#4ECDC4')
    ax5.plot(pred_denorm[:sample_size], label='Predicted', linewidth=2, alpha=0.8, color='#FF6B6B')
    ax5.set_xlabel('Sample', fontsize=11)
    ax5.set_ylabel('Price (USD)', fontsize=11)
    ax5.set_title(f'Predictions vs Actuals - Real Prices', fontsize=12, fontweight='bold')
    ax5.legend(fontsize=9)
    ax5.grid(True, alpha=0.3)
    ax5.fill_between(range(sample_size), actual_denorm[:sample_size],
                      pred_denorm[:sample_size], alpha=0.2)

    # 6. Scatter Plot
    ax6 = plt.subplot(3, 3, 6)
    ax6.scatter(actual_denorm, pred_denorm, alpha=0.4, s=15, c=np.arange(len(actual_denorm)),
                cmap='viridis', edgecolors='none')
    min_val = min(actual_denorm.min(), pred_denorm.min())
    max_val = max(actual_denorm.max(), pred_denorm.max())
    ax6.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2.5, label='Perfect prediction')
    ax6.set_xlabel('Actual Price', fontsize=11)
    ax6.set_ylabel('Predicted Price', fontsize=11)
    ax6.set_title('Prediction Scatter Plot', fontsize=12, fontweight='bold')
    ax6.legend(fontsize=9)
    ax6.grid(True, alpha=0.3)

    # 7. Error Distribution
    ax7 = plt.subplot(3, 3, 7)
    errors = pred_denorm - actual_denorm
    n, bins, patches = ax7.hist(errors, bins=60, edgecolor='black', alpha=0.7, color='#95E1D3')
    ax7.axvline(x=0, color='red', linestyle='--', linewidth=2.5, label='Zero error')
    ax7.axvline(x=errors.mean(), color='orange', linestyle='--', linewidth=2,
                label=f'Mean: {errors.mean():.6f}')
    ax7.set_xlabel('Prediction Error', fontsize=11)
    ax7.set_ylabel('Frequency', fontsize=11)
    ax7.set_title('Error Distribution', fontsize=12, fontweight='bold')
    ax7.legend(fontsize=9)
    ax7.grid(True, alpha=0.3, axis='y')

    # 8. Absolute Error Over Time
    ax8 = plt.subplot(3, 3, 8)
    abs_errors = np.abs(errors)
    ax8.plot(abs_errors[:sample_size], linewidth=1, alpha=0.4, color='gray')
    ax8.plot(smooth(abs_errors[:sample_size].tolist(), weight=0.9),
             linewidth=2.5, color='#FF6B6B', label='Smoothed')
    ax8.axhline(y=abs_errors.mean(), color='orange', linestyle='--',
                linewidth=2, label=f'Mean: {abs_errors.mean():.6f}')
    ax8.set_xlabel('Sample', fontsize=11)
    ax8.set_ylabel('Absolute Error', fontsize=11)
    ax8.set_title('Absolute Error Over Time', fontsize=12, fontweight='bold')
    ax8.legend(fontsize=9)
    ax8.grid(True, alpha=0.3)

    # 9. Percentage Error Distribution
    ax9 = plt.subplot(3, 3, 9)
    pct_errors = (errors / actual_denorm) * 100
    n, bins, patches = ax9.hist(pct_errors, bins=60, edgecolor='black', alpha=0.7, color='#F38181')
    ax9.axvline(x=0, color='red', linestyle='--', linewidth=2.5, label='Zero error')
    ax9.axvline(x=pct_errors.mean(), color='orange', linestyle='--', linewidth=2,
                label=f'Mean: {pct_errors.mean():.3f}%')
    ax9.set_xlabel('Percentage Error (%)', fontsize=11)
    ax9.set_ylabel('Frequency', fontsize=11)
    ax9.set_title('Percentage Error Distribution', fontsize=12, fontweight='bold')
    ax9.legend(fontsize=9)
    ax9.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/training_results.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\n📈 Gráficas generadas y guardadas exitosamente")
    print(f"📁 Ubicación: /content/drive/MyDrive/training_results.png")

# ============================================================
#     EXPORTACIÓN ONNX CON TORCH.JIT.TRACE (MÉTODO FUNCIONAL)
# ============================================================

def export_to_onnx_with_jit_trace(model, input_shape, output_path, device='cpu'):
    """
    Exportar modelo PyTorch a ONNX usando torch.jit.trace
    Este es el método más confiable para PyTorch
    """
    print(f"\n{'='*60}")
    print(f"   EXPORTANDO PYTORCH → ONNX CON JIT.TRACE")
    print(f"{'='*60}")
    print(f"📦 Ruta: {output_path}")

    try:
        model.eval()
        model.to(device)

        dummy_input = torch.randn(1, input_shape[0], input_shape[1], device=device)
        print(f"📐 Input shape: {dummy_input.shape}")

        print(f"⏳ Trazando modelo con torch.jit.trace...")
        with torch.no_grad():
            traced_model = torch.jit.trace(model, dummy_input)
        print(f"✅ Modelo trazado exitosamente")

        print(f"⏳ Exportando a ONNX...")

        # CORRECCIÓN: Usar dynamo=False para evitar el error de dynamic_axes
        torch.onnx.export(
            traced_model,
            dummy_input,
            output_path,
            export_params=True,
            opset_version=14,  # Versión más reciente y estable
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={
                'input': {0: 'batch_size'},
                'output': {0: 'batch_size'}
            },
            verbose=False,
            dynamo=False  # ← CLAVE: Deshabilitar dynamo para evitar error
        )

        if os.path.exists(output_path):
            size_mb = os.path.getsize(output_path) / (1024 * 1024)
            print(f"\n{'='*60}")
            print(f"✅✅✅ ¡EXPORTACIÓN EXITOSA! ✅✅✅")
            print(f"{'='*60}")
            print(f"📦 Archivo: {output_path}")
            print(f"📊 Tamaño: {size_mb:.2f} MB")
            print(f"{'='*60}\n")
            return True
        else:
            print(f"❌ Error: Archivo no se generó")
            return False

    except Exception as e:
        print(f"❌ Error: {e}")
        print(f"\n💡 Probando método alternativo sin dynamic_axes...")

        try:
            # Método alternativo sin dynamic_axes
            torch.onnx.export(
                traced_model,
                dummy_input,
                output_path,
                export_params=True,
                opset_version=14,
                input_names=['input'],
                output_names=['output'],
                verbose=False
            )

            if os.path.exists(output_path):
                size_mb = os.path.getsize(output_path) / (1024 * 1024)
                print(f"✅ Exportación exitosa (sin dynamic batch)")
                print(f"📦 Archivo: {output_path}")
                print(f"📊 Tamaño: {size_mb:.2f} MB\n")
                return True
        except Exception as e2:
            print(f"❌ Error en método alternativo: {e2}")
            return False

# ============================================================
#     EXPORTACIÓN CON TF2ONNX (CONVIRTIENDO PYTORCH → TF → ONNX)
# ============================================================

def export_to_onnx_via_tensorflow(model, input_shape, output_path, X_train, y_train,
                                   X_val, y_val, epochs=50, batch_size=128):
    """
    MÉTODO CON TF2ONNX: PyTorch → TensorFlow → ONNX

    Pasos:
    1. Crear modelo TensorFlow equivalente
    2. Entrenar o copiar pesos (entrenamiento recomendado)
    3. Exportar con tf2onnx.convert.from_keras()
    """
    print(f"\n{'='*60}")
    print(f"   EXPORTACIÓN PYTORCH → TENSORFLOW → ONNX")
    print(f"{'='*60}")

    try:
        # Instalar dependencias
        print(f"\n📦 Verificando dependencias...")
        try:
            import tensorflow as tf
            from tensorflow import keras
            import tf2onnx
            import onnx as onnx_lib
            print(f"✅ Dependencias disponibles")
        except ImportError:
            print(f"📦 Instalando dependencias...")
            import subprocess
            subprocess.check_call(['pip', 'install', '-q', 'tensorflow', 'tf2onnx', 'onnx'])
            import tensorflow as tf
            from tensorflow import keras
            import tf2onnx
            import onnx as onnx_lib
            print(f"✅ Dependencias instaladas")

        # 1. Crear modelo TensorFlow equivalente
        print(f"\n🧠 Creando modelo TensorFlow equivalente...")

        seq_length, num_features = input_shape
        hidden_size = model.hidden_size
        num_layers = model.num_layers

        tf_model = keras.Sequential(name='LSTM_Forex_TF')
        tf_model.add(keras.layers.InputLayer(input_shape=(seq_length, num_features)))

        # Añadir capas LSTM
        for i in range(num_layers):
            return_seq = (i < num_layers - 1)
            tf_model.add(keras.layers.LSTM(
                hidden_size,
                return_sequences=return_seq,
                dropout=0.2 if num_layers > 1 else 0.0,
                name=f'lstm_{i+1}'
            ))

        # Capas fully connected
        tf_model.add(keras.layers.Dense(hidden_size // 2, activation='relu', name='fc1'))
        tf_model.add(keras.layers.Dropout(0.2, name='dropout'))
        tf_model.add(keras.layers.Dense(1, name='output'))

        tf_model.compile(
            optimizer=keras.optimizers.Adam(0.001),
            loss='mse',
            metrics=['mae']
        )

        print(f"✅ Modelo TensorFlow creado")
        tf_model.summary()

        # 2. Entrenar modelo TensorFlow
        print(f"\n🏋️ Entrenando modelo TensorFlow...")
        print(f"   Epochs: {epochs}")
        print(f"   Batch size: {batch_size}")

        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                verbose=1
            )
        ]

        history = tf_model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )

        print(f"\n✅ Entrenamiento TensorFlow completado")

        # 3. CORRECCIÓN: Guardar y recargar modelo para evitar error de output_names
        print(f"\n💾 Guardando modelo temporal...")
        temp_model_path = "/tmp/temp_tf_model.keras"
        tf_model.save(temp_model_path)

        print(f"📦 Recargando modelo...")
        tf_model_reloaded = keras.models.load_model(temp_model_path)

        # 4. Exportar con tf2onnx (ESTE ES EL CÓDIGO QUE QUERÍAS)
        print(f"\n📦 Exportando TensorFlow → ONNX con tf2onnx...")
        print(f"   Destino: {output_path}")

        # MÉTODO CORRECTO: Especificar input_signature explícitamente
        spec = (tf.TensorSpec((None, seq_length, num_features), tf.float32, name="input"),)

        # AQUÍ ESTÁ TU LÍNEA DE CÓDIGO (CORREGIDA)
        onnx_model, _ = tf2onnx.convert.from_keras(
            tf_model_reloaded,
            input_signature=spec,
            opset=13,
            output_path=output_path
        )

        # Verificar archivo
        if os.path.exists(output_path):
            size_mb = os.path.getsize(output_path) / (1024 * 1024)

            print(f"\n{'='*60}")
            print(f"✅✅✅ ¡EXPORTACIÓN TF2ONNX EXITOSA! ✅✅✅")
            print(f"{'='*60}")
            print(f"📦 Archivo: {output_path}")
            print(f"📊 Tamaño: {size_mb:.2f} MB")

            # Validar con ONNX
            try:
                onnx_model_loaded = onnx_lib.load(output_path)
                onnx_lib.checker.check_model(onnx_model_loaded)
                print(f"✅ Validación ONNX: Modelo válido")
                print(f"   Inputs: {[i.name for i in onnx_model_loaded.graph.input]}")
                print(f"   Outputs: {[o.name for o in onnx_model_loaded.graph.output]}")
            except Exception as e:
                print(f"⚠️ Validación: {e}")

            print(f"{'='*60}\n")
            return True, tf_model_reloaded
        else:
            print(f"❌ Error: Archivo no se generó")
            return False, None

    except Exception as e:
        print(f"\n❌ Error en exportación TF2ONNX: {e}")
        print(f"\n💡 Detalles del error:")
        import traceback
        traceback.print_exc()
        return False, None

def ensure_directory_exists(file_path):
    """Crear directorio si no existe"""
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
        print(f"📁 Directorio creado: {directory}")

# ============================================================
#     FUNCIÓN PRINCIPAL - MODIFICADA PARA 6H
# ============================================================

def main():
    print("=" * 60)
    print("   LSTM FOREX - MODELO DAILY (DIARIO)")
    print("=" * 60)

    # Configuración
    data_path = "/content/drive/MyDrive/EURUSD_1d_data.csv"  # ← CAMBIO
    onnx_path = "/content/drive/MyDrive/FOREX_MODELS/eurusd_lstm_daily.onnx"  # ← CAMBIO

    # Secuencia: 60 días = 2 meses de datos históricos
    SEQUENCE_LENGTH = 60  # ← 60 días (antes eran 60 períodos de 6h)
    FEATURES = ['open', 'high', 'low', 'close']
    HIDDEN_SIZE = 256
    NUM_LAYERS = 3
    DROPOUT = 0.2
    BATCH_SIZE = 128
    EPOCHS = 150
    LEARNING_RATE = 0.001
    EARLY_STOPPING_PATIENCE = 20

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n🖥️ Device: {device}")

 # 1. Cargar datos DAILY directamente (NO CONVERTIR)
    print(f"\n{'='*60}")
    print(f"CARGANDO DATOS DAILY")
    print(f"{'='*60}")
    df = pd.read_csv(data_path)
    df['time'] = pd.to_datetime(df['time'])
    print(f"✅ {len(df):,} registros daily cargados")

    # Verificar cantidad mínima
    if len(df) < 5000:
        print(f"\n⚠️ ADVERTENCIA: Datos insuficientes")
        print(f"   Tienes: {len(df):,} datos daily")
        print(f"   Necesitas: ~10,000 datos daily")
        print(f"   Equivale a: ~27 años de datos")
        user_input = 's' #input("\n¿Deseas continuar de todas formas? (s/n): ")
        if user_input.lower() != 's':
            print("Proceso cancelado")
            return

    # 3. Preparar datos
    print(f"\n{'='*60}")
    print(f"PREPARANDO DATOS")
    print(f"{'='*60}")
    X, y, scaler = prepare_data(df, SEQUENCE_LENGTH, FEATURES)

    # 4. Dividir datos
    print(f"\n{'='*60}")
    print(f"DIVIDIENDO DATOS")
    print(f"{'='*60}")
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, shuffle=False)

    print(f"📊 División:")
    print(f"   Train: {len(X_train):,}")
    print(f"   Val:   {len(X_val):,}")
    print(f"   Test:  {len(X_test):,}")

    # 5. Crear DataLoaders
    train_dataset = ForexDataset(X_train, y_train)
    val_dataset = ForexDataset(X_val, y_val)
    test_dataset = ForexDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 6. Crear y entrenar modelo
    print(f"\n{'='*60}")
    print(f"CREANDO Y ENTRENANDO MODELO")
    print(f"{'='*60}")

    model = ImprovedLSTMModel(
        input_size=len(FEATURES),
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        output_size=1,
        dropout=DROPOUT
    )

    total_params = sum(p.numel() for p in model.parameters())
    print(f"🧠 Arquitectura:")
    print(f"   Hidden size: {HIDDEN_SIZE}")
    print(f"   Num layers: {NUM_LAYERS}")
    print(f"   Total parámetros: {total_params:,}")
    print(f"   ⏰ Predicción: Cada 24 horas")

    train_losses, val_losses, learning_rates = train_model(
        model, train_loader, val_loader,
        epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        device=device,
        early_stopping_patience=EARLY_STOPPING_PATIENCE
    )

    # 7. Evaluar modelo
    print(f"\n{'='*60}")
    print(f"EVALUANDO MODELO")
    print(f"{'='*60}")

    predictions, actuals, metrics, pred_denorm, actual_denorm = evaluate_model(
        model, test_loader, scaler, device
    )

    # 8. Visualizar resultados
    print(f"\n{'='*60}")
    print(f"GENERANDO VISUALIZACIONES")
    print(f"{'='*60}")

    plot_comprehensive_results(
        train_losses, val_losses, learning_rates,
        predictions, actuals, pred_denorm, actual_denorm
    )


    # 9. Exportar a ONNX
    print(f"\n{'='*60}")
    print(f"EXPORTACIÓN A ONNX")
    print(f"{'='*60}")

    # ✅ AGREGAR ESTO ANTES DE EXPORTAR
    ensure_directory_exists(onnx_path)

    success = export_to_onnx_with_jit_trace(
        model=model,
        input_shape=(SEQUENCE_LENGTH, len(FEATURES)),
        output_path=onnx_path,
        device=device
    )

    # 10. Guardar configuración y scaler
    print(f"\n{'='*60}")
    print(f"GUARDANDO CONFIGURACIÓN")
    print(f"{'='*60}")

    scaler_path = "/content/drive/MyDrive/FOREX_MODELS/eurusd_scaler_daily.pkl"

    # ✅ AGREGAR ESTO ANTES DE GUARDAR SCALER
    ensure_directory_exists(scaler_path)
    joblib.dump(scaler, scaler_path)
    print(f"✅ Scaler guardado: {scaler_path}")

    config = {
        'timeframe': 'DAILY',
        'sequence_length': SEQUENCE_LENGTH,
        'features': FEATURES,
        'hidden_size': HIDDEN_SIZE,
        'num_layers': NUM_LAYERS,
        'metrics': metrics,
        'notes': 'Modelo entrenado con datos diarios. Secuencia de 60 días = 2 meses'
    }

    config_path = "/content/drive/MyDrive/FOREX_MODELS/eurusd_model_config_daily.json"

    # ✅ AGREGAR ESTO ANTES DE GUARDAR CONFIG
    ensure_directory_exists(config_path)
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"✅ Config guardado: {config_path}")

    # Guardar también modelo PyTorch
    torch_path = "/content/drive/MyDrive/FOREX_MODELS/eurusd_lstm_daily.pth"

    # ✅ AGREGAR ESTO ANTES DE GUARDAR PYTORCH MODEL
    ensure_directory_exists(torch_path)
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': None,
        'metrics': metrics,
        'config': config
    }, torch_path)
    print(f"✅ Modelo PyTorch guardado: {torch_path}")


    total_time = time.time() - start_total

    print(f"\n{'='*60}")
    print(f"✅✅✅ ¡COMPLETADO! ✅✅✅")
    print(f"{'='*60}")
    print(f"📦 Modelo PyTorch: {torch_path}")
    print(f"📦 ONNX: {onnx_path}")
    print(f"💾 Scaler: {scaler_path}")
    print(f"⚙️ Config: {config_path}")
    print(f"⏱️ Tiempo total: {total_time/60:.2f} min")

    print(f"⏰ Timeframe: DAILY (DIARIO)")  # ← CAMBIO
    print(f"📊 Datos usados: {len(df):,} velas diarias")  # ← CAMBIO

    print(f"\n💡 RESUMEN:")
    print(f"   - Datos diarios descargados directamente")  # ← CAMBIO
    print(f"   - Modelo predice precio cada día")  # ← CAMBIO
    print(f"   - Secuencia de {SEQUENCE_LENGTH} días = {SEQUENCE_LENGTH/30:.1f} meses")  # ← CAMBIO
    print(f"   - Para 10k datos daily necesitas ~27 años de historia")  # ← CAMBIO

if __name__ == "__main__":
    main()


   LSTM FOREX - MODELO DAILY (DIARIO)

🖥️ Device: cpu

CARGANDO DATOS DAILY
✅ 5,721 registros daily cargados

PREPARANDO DATOS

📊 Preparando datos...
   Features: ['open', 'high', 'low', 'close']
   Longitud de secuencia: 60
   ⏳ Normalizando datos con MinMaxScaler...
   ✅ Normalización completada
   📐 Rango: [0.0000, 1.0000]
   ⏳ Creando secuencias temporales...


/tmp/ipython-input-339678726.py:698: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time'] = pd.to_datetime(df['time'])


   Secuencias:   0%|          | 0/5661 [00:00<?, ?it/s]


✅ Datos preparados:
   Shape X: (5661, 60, 4)
   Shape y: (5661,)

DIVIDIENDO DATOS
📊 División:
   Train: 4,329
   Val:   765
   Test:  567

CREANDO Y ENTRENANDO MODELO
🧠 Arquitectura:
   Hidden size: 256
   Num layers: 3
   Total parámetros: 1,353,985
   ⏰ Predicción: Cada 24 horas

🏋️ Entrenando modelo...
   Epochs: 150
   Learning rate: 0.001
   Device: cpu
   Early stopping patience: 20


📄 Epochs:   0%|          | 0/150 [00:00<?, ?it/s]

...

   ⚠️  EarlyStopping counter: 4/20


  📚 Train 47/150:   0%|          | 0/34 [00:00<?, ?it/s]

  ✓ Val 47/150:   0%|          | 0/6 [00:00<?, ?it/s]

   ⚠️  EarlyStopping counter: 5/20


  📚 Train 48/150:   0%|          | 0/34 [00:00<?, ?it/s]